In [1]:
# @title Installs and imports (run me first!)
!pip install pyglet~=1.3.2
!apt install -y graphviz
!pip install flax
!pip install graphviz
!pip install pyvirtualdisplay
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install 'gym[atari]'
!pip install -U dopamine-rl
!pip install dopamine-rl
!pip install cmake 'gym[atari]' scipy


import flax
import random
from graphviz import Digraph
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import gym
from IPython.display import clear_output
from time import sleep

from IPython.display import HTML
from pprint import pprint
import logging
from pyvirtualdisplay import Display
logging.getLogger("pyvirtualdisplay").setLevel(logging.ERROR)

display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.42.2-3build2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.2/214.2 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 KB 900.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.9/dist-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(


In [2]:
random.seed(1234)

env = gym.make("Taxi-v3").env

env.reset()
env.render()

/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't 

In [3]:
#state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
#print("State:", state)

#env.s = state

#env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


#Solving the environment without Reinforcement Learning

In [8]:
#env.s = 328  # set environment to illustration's state
trip_length = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    trip_length += 1
    
    
print("Timesteps taken: {}".format(trip_length))
print("Penalties: {}".format(penalties))

Timesteps taken: 1517
Penalties: 436


In [9]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1517
State: 85
Action: 5
Reward: 20


#Implementing Q-learning

In [13]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

q_table = np.zeros([env.observation_space.n, env.action_space.n])
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100000):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 99900
Training finished.

CPU times: user 1min 32s, sys: 7.56 s, total: 1min 39s
Wall time: 1min 35s


In [11]:
"""Evaluate agent's performance after Q-learning"""

total_trip_length, total_penalties = 0, 0
episodes = 10

for tripnum in range(episodes):
    state = env.reset()
    trip_length, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        #action = env.action_space.sample()
        state, reward, done, info = env.step(action)

        #clear_output(wait=True)
        #print("Trip number " + str(tripnum) + " Step " + str(trip_length))
        #print(env.render(mode='ansi'))
        #sleep(.1)

        if reward == -10:
            penalties += 1

        trip_length += 1

    total_penalties += penalties
    total_trip_length += trip_length

print(f"Results after {episodes} episodes:")
print(f"\tAverage timesteps per episode: {total_trip_length / episodes}")
print(f"\tAverage penalties per episode: {total_penalties / episodes}")

Results after 10 episodes:
	Average timesteps per episode: 12.2
	Average penalties per episode: 0.0


In [14]:
"""Evaluate agent's performance after Q-learning"""

total_trip_length, total_penalties = 0, 0
episodes = 10

for tripnum in range(episodes):
    state = env.reset()
    trip_length, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        #action = env.action_space.sample()
        state, reward, done, info = env.step(action)

        clear_output(wait=True)
        print("Trip number " + str(tripnum) + " Step " + str(trip_length))
        print(env.render(mode='ansi'))
        sleep(.3)

        if reward == -10:
            penalties += 1

        trip_length += 1

    total_penalties += penalties
    total_trip_length += trip_length

print(f"Results after {episodes} episodes:")
print(f"\tAverage timesteps per episode: {total_trip_length / episodes}")
print(f"\tAverage penalties per episode: {total_penalties / episodes}")

Trip number 9 Step 15
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Results after 10 episodes:
	Average timesteps per episode: 14.2
	Average penalties per episode: 0.0
